In [76]:
import pyomo.environ as pyo
import pandas as pd
import json
import math
import statistics
import re

In [81]:
# Parameters
boards = 10
balance = 0.85
friend_cost = 1
avoid_cost = -5
rating_threshold = 10
rating_viol_cost = 1

In [46]:
class Player:
    pref_score = 0
    team = None
    board = None
    req_met = False

    def __init__(self, name, rating, friends, avoid, date, alt, previous_season_alt):
        self.name = name
        self.rating = rating
        self.friends = friends
        self.avoid = avoid
        self.date = date
        self.alt = alt
        self.previous_season_alt = previous_season_alt

    @classmethod
    def player_from_json(cls, player):
        return cls(
            player['name'],
            player['rating'],
            player['friends'],
            player['avoid'],
            player['date_created'],
            player['prefers_alt'],
            player.get('previous_season_alternate', False)
        )

    def __repr__(self):
        return str((self.name, self.board, self.rating, self.req_met))

    def __lt__(self, other):
        return True

    def set_pref_score(self):
        self.pref_score = 0
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.pref_score += 1
            else:
                self.pref_score -= 1
        for avoid in self.avoid:
            if avoid in self.team.get_boards():
                self.pref_score -= 1
        # player with more than 5 choices can be <5 preference even if all teammates are preferred

    def set_req_met(self):
        self.req_met = False
        if not self.friends:
            self.req_met = None
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.req_met = True


class Team:
    def __init__(self, boards):
        self.boards = [None for x in range(boards)]

    def __str__(self):
        return str((self.boards, self.team_pref_score, self.get_mean()))

    def __repr__(self):
        return "Team:{0}".format(id(self))

    def __lt__(self, other):
        return True

    def changeBoard(self, board, new_player):
        # updates the player on a board and updates that player's team attribute
        if self.boards[board]:
            self.boards[board].team = None
        self.boards[board] = new_player
        if new_player.team:
            new_player.team.boards[board] = None
        new_player.team = self

    def get_mean(self, expected_rating=False):
        # expected_rating is an unused parameter in this version.
        # it is used by the tournament.models.Team.get_mean method.
        ratings = [board.rating for board in self.boards]
        mean = sum(ratings) / len(ratings)
        return mean

    def get_boards(self):
        return self.boards

    def get_player(self, board):
        return self.boards[board]

    def set_team_pref_score(self):
        self.team_pref_score = sum([x.pref_score for x in self.boards])
        
def split_into_equal_groups_by_rating(players, group_number):
    players.sort(key=lambda player: player.rating, reverse=True)
    avg = len(players) / float(group_number)
    players_split = []
    last = 0.0
    while round(last) < len(players):
        players_split.append(players[int(round(last)):int(round(last + avg))])
        last += avg
    return players_split


def get_rating_bounds_of_split(split):
    min_ratings = [min([p.rating for p in board]) for board in split]
    max_ratings = [max([p.rating for p in board]) for board in split]
    min_ratings[-1] = 0
    max_ratings[0] = 5000
    return list(zip(min_ratings, max_ratings))

def flatten(lst):
    return [item for sub_lst in lst for item in sub_lst]

In [47]:
# Load data
with open("data.json") as f:
    playerdata = json.load(f)

df = pd.DataFrame(playerdata)

In [48]:
players = []
for player in playerdata:
    if player['has_20_games'] and player['in_slack']:
        players.append(Player.player_from_json(player))
    else:
        pass
        # print("{0} skipped".format(player['name']))
players.sort(key=lambda player: player.rating, reverse=True)

# Split into those that want to be alternates vs those that do not.
alternates = [p for p in players if p.alt]
players = [p for p in players if not p.alt]

# splits list of Player objects into near equal lists, sectioned by rating
players_split = split_into_equal_groups_by_rating(players, boards)
team_rating_bounds = get_rating_bounds_of_split(players_split)

num_teams = int(math.ceil((len(players_split[0]) * balance) / 2.0) * 2)
# print(f"Targetting {num_teams} teams")

# separate latest joining players into alternate lists as required
for n, board in enumerate(players_split):
    board.sort(key=lambda player: (0 if player.previous_season_alt else 1, player.date))
    alternates.extend(board[num_teams:])
    del board[num_teams:]
    board.sort(key=lambda player: player.rating, reverse=True)

alts_split = split_into_equal_groups_by_rating(alternates, boards)
alt_rating_bounds = get_rating_bounds_of_split(alts_split)

players = flatten(players_split)

# print len(players)
# print num_teams
# print alts_split

for n, board in enumerate(players_split):
    for player in board:
        player.board = n

def convert_name_list(string_of_names, players):
    pattern = r"([^-_a-zA-Z0-9]|^){0}([^-_a-zA-Z0-9]|$)"
    return [player for player in players
            if re.search(pattern.format(player.name), string_of_names, flags=re.I)]

for player in players:
    filtered_players = [p for p in players if p.board != player.board]
    player.friends = convert_name_list(player.friends, filtered_players)
    player.avoid = convert_name_list(player.avoid, filtered_players)

In [80]:
target_team_rating = statistics.mean([player.rating for player in players])
target_team_rating

1865.0575

In [82]:
# Model
model = pyo.AbstractModel()

# Sets
model.players = pyo.Set(initialize=[player.name for player in players])
model.boards = pyo.Set(initialize=[board_num for board_num in range(boards)])
model.teams = pyo.Set(initialize=[team_num for team_num in range(num_teams)])

# Parameters
model.param_player_rating = pyo.Param(model.players, initialize=[player.rating for player in players])
model.param_player_rd = pyo.Param(model.players, initialize=[65 for player in players])
model.param_player_expscore = pyo.Param(model.players, initialize=[4.0 for player in players])
model.param_player_boards = pyo.Param(model.players, initialize=[player.board for player in players])
model.param_friend_cost = pyo.Param(initialize=friend_cost)
model.param_avoid_cost = pyo.Param(initialize=avoid_cost)
model.param_rating_viol_cost = pyo.Param(initialize=rating_viol_cost)

# Decision variables
model.dvar_x = pyo.Var(model.players, model.boards, model.teams, within=pyo.Binary)

# Soft constraint penalty decision variables
model.pen_friends = pyo.Var(model.players, model.teams, within=pyo.Binary)
model.pen_avoids = pyo.Var(model.players, model.teams, within=pyo.Binary)
model.pen_balance = pyo.Var(model.teams, within=pyo.NonNegativeReals)

# Objective
def objective_function(model):
    return model.param_friend_cost*pyo.summation(model.pen_friends) \
    + model.param_avoid_cost*pyo.summation(model.pen_avoids) \
    + model.param_rating_viol_cost*pyo.summation(model.pen_balance)
model.objective = pyo.Objective(rule=objective_function, sense=pyo.maximize)

# Hard Constraint: Each player must be on one and only one team
def constraint_one_team(model, p):
    return sum(model.dvar_x[p, model.boards, model.teams]) == 1
model.cons_one_team = pyo.Constraint(model.players, rule=constraint_one_team)

# Hard Constraint: Each player must be on their own board
def constraint_own_board(model, p):
    return sum(model.dvar_x[p, model.player_boards[p], model.teams]) == 1
model.cons_own_board = pyo.Constraint(model.players, rule=constraint_own_board)

# Soft Constraint: Teams must be balanced
def constraint_team_balance_rating_lb(model, t):
    return sum(model.param_player_rating[model.p]*model.dvar_x[model.p, model.boards, t]) \
    >= target_team_rating - rating_threshold - model.pen_balance[t]
model.cons_team_balance_rating_lb = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_lb)

def constraint_team_balance_rating_ub(model, t):
    return sum(model.param_player_rating[model.p]*model.dvar_x[model.p, model.boards, t]) \
    <= target_team_rating + rating_threshold + model.pen_balance[t]
model.cons_team_balance_rating_ub = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_ub)
